In [1]:
from utils.trainingUtils import TrainingUtils
import numpy as np
import os

2023-04-03 09:25:19.812310: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 09:25:20.877400: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [2]:
working_dir = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test2/pre_processing/npy"
original_path = os.path.join(working_dir, "original.npy")
npy_path = os.path.join(working_dir, "mask.npy")

In [3]:
print(os.listdir(working_dir))

['mask.npy', 'test.npy', 'original.npy']


In [4]:
images = TrainingUtils.load_file(original_path)
labels = TrainingUtils.load_file(npy_path)

In [5]:
images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)
labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)

In [6]:
perm = TrainingUtils.random_permutation(len(images))

In [7]:
print(perm)

[1531 1471 1170 ... 1020  810  631]


In [8]:
images = TrainingUtils.randomize_data(images, perm)
labels = TrainingUtils.randomize_data(labels, perm)

In [9]:
images = TrainingUtils.individual_normalize(images)
labels = labels.astype(np.float64)

In [10]:
X_train, X_val, X_test = TrainingUtils.train_val_test_split(images, 1200, 1700)
y_train, y_val, y_test = TrainingUtils.train_val_test_split(labels, 1200, 1700)

In [11]:
model = TrainingUtils.init_model()

2023-04-03 09:25:46.575550: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-04-03 09:25:46.575594: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: chimera13
2023-04-03 09:25:46.575598: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: chimera13
2023-04-03 09:25:46.575681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 450.172.1
2023-04-03 09:25:46.575694: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 450.172.1
2023-04-03 09:25:46.575697: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 450.172.1


In [12]:
model = TrainingUtils.compile_model(model)

In [ ]:
history = TrainingUtils.training(model, X_train, y_train, X_val, y_val, batch=60, epochs=40)

training and prediction for batch size:   60 epochs:   40


In [ ]:
TrainingUtils.plot_predict(history, model, X_test, y_test)